# 🧪 Ablation Study: YOLOv8 to GSO-YOLO on SODA Dataset

In this notebook, we conduct an ablation study to evaluate the impact of integrating the **GSO-YOLO modules** into YOLOv8.  
We use the **SODA dataset** and progressively test:

1. **GOM** – Global Optimization Module (GAM-based attention)  
2. **SCM** – Steady Capture Module (temporal smoothing)  
3. Combined **GOM + SCM** configuration (**GSO-YOLO**)

Each variant is compared to the **baseline YOLOv8** to measure performance improvements in terms of mAP.

📦 Base repo: [Ultralytics YOLOv8](https://github.com/ultralytics/ultralytics)  
🧠 Experiment goal: Quantify the contribution of each GSO-YOLO module and their combination.

# Results Analysis

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/DISAL/hazard-detection-project')

from utils.visualizations.inference_results import compare_inference_results

In [ ]:
results_df = compare_inference_results(
    "/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/evaluations/baseline_yolov8n/results/baseline_yolov8n.json",
    "/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/evaluations/baseline_yolov8n/results/gam_yolov8n.json",
    "/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/evaluations/baseline_yolov8n/results/baseline_yolov8x.json",
    "/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/evaluations/baseline_yolov8n/results/gam_yolov8x.json",

    output_dir="/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/evaluations/plots"
)

# Implementation

## 📁 1. Setup

Clone our fork of Ultralytics with the YOLOv8 variants implemented and install dependencies.

```
!git clone https://github.com/DoniaGasmii/gso-yolo-v8.git
```


In [ ]:
# Mount Google Drive (dataset and yolov8-gso.yaml are there)
from google.colab import drive
drive.mount('/content/drive')

In [3]:
%cd /content/drive/MyDrive/DISAL/gso-yolo-v8


/content/drive/MyDrive/DISAL/gso-yolo-v8


In [4]:
!pip install -e .  # install in editable mode

Obtaining file:///content/drive/MyDrive/DISAL/gso-yolo-v8
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━

In [5]:
# We uninstall albumentations because sometimes during training setting augment to False doesn't disable them
%pip uninstall -y albumentations

## 📦 2. Load Dataset: SODA

In this section, we load the SODA dataset. It should be in YOLO format with the following structure:
```
datasets/SODA/
├── images/
│ ├── train/
│ ├── val/
├── labels/
│ ├── train/
│ ├── val/

```

In [6]:
data_path = "/content/drive/MyDrive/DISAL/Disal-Data/Benchmarks/SODA/data.yaml"
save_dir = "/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo"

## ⚙️ 3. Train YOLOv8 (Baseline)

We start by training the original YOLOv8 model using the `yolov8.yaml` configuration on the SODA dataset.

In [ ]:
!yolo detect train \
  model=yolov8.yaml \
  data={data_path} \
  val=False \
  epochs=100 \
  batch=-1  \
  imgsz=640 \
  lr0=0.01 \
  weight_decay=0.0005 \
  momentum=0.937 \
  project={save_dir} \
  name=baseline_yolov8n \
  augment=False   auto_augment=False

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
New https://pypi.org/project/ultralytics/8.3.173 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=False, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/DISAL/Disal-Data/Benchmarks/SODA/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, 

## ⚙️ 4. Train YOLOv8 + GAM

Now we train the modified YOLOv8 model that includes the **GAM module**, using the `yolov8-gso.yaml` config file.


In [ ]:
%pip uninstall -y albumentations

Found existing installation: albumentations 2.0.8
Uninstalling albumentations-2.0.8:
  Successfully uninstalled albumentations-2.0.8


In [ ]:
!yolo detect train \
  model=yolov8-gam.yaml \
  data={data_path} \
  val=False \
  epochs=100 \
  batch=-1  \
  imgsz=640 \
  lr0=0.01 \
  weight_decay=0.0005 \
  momentum=0.937 \
  project={save_dir} \
  name=gam_yolov8n \
  augment=False   auto_augment=False

WARNING ⚠️ no model scale passed. Assuming scale='n'.
New https://pypi.org/project/ultralytics/8.3.173 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=False, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/DISAL/Disal-Data/Benchmarks/SODA/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8-gam.yaml,

## ⚙️ 5. Train YOLOv8x


In [ ]:
!yolo detect train \
  model=yolov8x.yaml \
  data={data_path} \
  val=False \
  epochs=100 \
  batch=-1  \
  imgsz=640 \
  lr0=0.01 \
  weight_decay=0.0005 \
  momentum=0.937 \
  project={save_dir} \
  name=baseline_yolov8x \
  augment=False   auto_augment=False

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
New https://pypi.org/project/ultralytics/8.3.174 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=False, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/DISAL/Disal-Data/Benchmarks/SODA/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, f

## ⚙️ 6. Train YOLOv8x + GAM


In [ ]:
!yolo detect train \
  model=yolov8x-gam.yaml \
  data={data_path} \
  val=False \
  epochs=100 \
  batch=-1  \
  imgsz=640 \
  lr0=0.01 \
  weight_decay=0.0005 \
  momentum=0.937 \
  project={save_dir} \
  name=gam_yolov8x \
  augment=False   auto_augment=False

New https://pypi.org/project/ultralytics/8.3.174 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=False, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/DISAL/Disal-Data/Benchmarks/SODA/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x-gam.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=

# Model Evaluation


In [ ]:
# Evaluate baseline_yolov8n
!yolo detect val model=/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/baseline_yolov8n/weights/best.pt \
    data={data_path} \
    project=/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/evaluations \
    name=baseline_yolov8n

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8 summary (fused): 72 layers, 3,008,573 parameters, 0 gradients, 8.1 GFLOPs
100% 755k/755k [00:00<00:00, 21.6MB/s]
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 1.3±0.5 MB/s, size: 951.5 KB)
val: Scanning /content/drive/MyDrive/DISAL/Disal-Data/Benchmarks/SODA/labels/val.cache... 17859 images, 50 backgrounds, 0 corrupt: 100% 17859/17859 [00:00<?, ?it/s]
val: /content/drive/MyDrive/DISAL/Disal-Data/Benchmarks/SODA/images/val/hd1729.jpg: 1 duplicate labels removed
val: /content/drive/MyDrive/DISAL/Disal-Data/Benchmarks/SODA/images/val/hd1821.jpg: 1 duplicate labels removed
val: /

In [ ]:
# Evaluate gam_yolov8n
!yolo detect val model=/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/gam_yolov8n/weights/best.pt \
    data={data_path} \
    project=/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/evaluations \
    name=gam_yolov8n

In [ ]:
# Evaluate baseline_yolov8x
!yolo detect val model=/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/baseline_yolov8x/weights/best.pt \
    data={data_path} \
    project=/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/evaluations \
    name=baseline_yolov8x


In [ ]:
# Evaluate gam_yolov8x
!yolo detect val model=/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/gam_yolov8x/weights/best.pt \
    data={data_path} \
    project=/content/drive/MyDrive/DISAL/Disal-Data/runs/detect/gso-yolo/evaluations \
    name=gam_yolov8x